《设计模式：可复用面向对象软件的基础》一书讲解迭代器设计模式时，在“适用性”一 节中说：
迭代器模式可用来：
* 访问一个聚合对象的内容而无需暴露它的内部表示
* 支持对聚合对象的多种遍历
* 为遍历不同的聚合结构提供一个统一的接口（即支持多态迭代）

为了“支持多种遍历”，必须能从同一个可迭代的实例中获取多个**独立**的迭代器，而且各个迭代器要能维护自身的内部状态，因此这一模式正确的实现方式是，每次调用 `iter(my_iterable)` 都新建一个独立的迭代器。这就是为什么这个示例需要定义 `SentenceIterator` 类。

所以，不应该把 Sentence 本身作为一个迭代器，否则每次调用 `iter(sentence)` 时返回的都是自身，就无法进行多次迭代了。


In [ ]:
import re


class Sentence:
    def  __init__(self, sentence):
        self.sentence = sentence
        self.words = re.findall(r'\w+', sentence)

    def __iter__(self):
        """返回 iter(self) 的结果"""
        return SentenceIterator(self.words)
        

class SentenceIterator:
    def __init__(self, words):
        self.words = words
        self._index = 0

    def __next__(self):
        """调用时返回下一个对象"""
        try:
            word = self.words[self._index]
        except IndexError:
            raise StopIteration()
        else:
            self._index += 1
        
        return word


    
sentence = Sentence('Return a list of all non-overlapping matches in the string.')
for word in sentence:
    print(word, end='·')

In [ ]:
# 使用生成器函数来帮我们创建迭代器
import re


class Sentence:
    def  __init__(self, sentence):
        self.sentence = sentence
        self.words = re.findall(r'\w+', sentence)

    def __iter__(self):
        for word in self.words:
            yield word
        return

sentence = Sentence('Return a list of all non-overlapping matches in the string.')
for word in sentence:
    print(word, end='·')

In [ ]:
# 使用 re.finditer 来惰性生成值
# 使用生成器表达式（很久没用过了）
import re


class Sentence:
    def  __init__(self, sentence):
        self.re_word = re.compile(r'\w+')
        self.sentence = sentence

    def __iter__(self):
        return (match.group()
                for match in self.re_word.finditer(self.sentence))

sentence = Sentence('Return a list of all non-overlapping matches in the string.')
for word in sentence:
    print(word, end='·')

In [ ]:
# 实用模块
import itertools

# takewhile & dropwhile
print(list(itertools.takewhile(lambda x: x < 3, [1, 5, 2, 4, 3])))
print(list(itertools.dropwhile(lambda x: x < 3, [1, 5, 2, 4, 3])))
# zip
print(list(zip(range(5), range(3))))
print(list(itertools.zip_longest(range(5), range(3))))

# itertools.groupby
animals = ['rat', 'bear', 'duck', 'bat', 'eagle', 'shark', 'dolphin', 'lion']
# groupby 需要假定输入的可迭代对象已经按照分组标准进行排序（至少同组的元素要连在一起）
print('----')
for length, animal in itertools.groupby(animals, len):
    print(length, list(animal))
print('----')
animals.sort(key=len)
for length, animal in itertools.groupby(animals, len):
    print(length, list(animal))
print('---')
# tee
g1, g2 = itertools.tee('abc', 2)
print(list(zip(g1, g2)))

`iter` 函数还有一个鲜为人知的用法：传入两个参数，使用常规的函数或任何可调用的对象创建迭代器。这样使用时，第一个参数必须是可调用的对象，用于不断调用（没有参数），产出各个值；第二个值是哨符，这是个标记值，当可调用的对象返回这个值时，触发迭代器抛出 StopIteration 异常，而不产出哨符。

In [ ]:
# iter 的神奇用法
# iter(callable, sentinel)
import random

def rand():
    return random.randint(1, 6)
# 不停调用 rand(), 直到产出一个 5
print(list(iter(rand, 5)))